# Analysing Fires data from NASA FIRMS

## About NASA FIRMS

The **Fire Information for Resource Management System (FIRMS)** provides access, with minimal delay, to satellite imagery, **active fire/hotspots**, and related products to identify the **location** and **intensity** of wildfire activity. FIRMS tools and applications provide geospatial data, products and services to support the broader fire management community, and to inform the general public. Global data are available within 3 hours of satellite observation; U.S. and Canada active fire detections are available in real-time.



## What dataset it provides ?

Fires data from below satellites
1. MODIS (since 2000) at 1km spatial resolution
2. SNPP VIIRS (since 2012) at 375m spatial resolution
3. NOAA VIIRS (since 2021) at 375 spatial resolution


## Getting data from NASA FIRMS

Just uncomment the below "wget command" and run the cell

In [ ]:
# let's download VIIRS SNPP fires data for United States of America for 2019

!wget https://firms.modaps.eosdis.nasa.gov/data/country/viirs-snpp/2019/viirs-snpp_2019_United_States.csv

## Load the data

In [ ]:
# !pip3 install rasterio -q
!apt install gdal-bin python3-gdal --quiet

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.stats import mode
from osgeo import gdal
import matplotlib.pyplot as plt
from shapely.geometry import Point
%matplotlib inline

In [ ]:
df = pd.read_csv("https://firms.modaps.eosdis.nasa.gov/data/country/viirs-snpp/2019/viirs-snpp_2019_United_States.csv")
df.head()

In [ ]:
# plotting fires based on when they were captured and their confidence flag
df.groupby(["daynight"])["confidence"].value_counts().plot(kind="bar")

* Please note use of "confidence" flag are subjective for example if you're working with agricultural sector most of the crop fires fall within nominal and low confidence. While if you're focussed on major wildfires they fall between high and nominal confidence.

## Monitoring fires over time

In [ ]:
# Convert the 'datetime' column to a datetime object if it's not already
df['datetime'] = pd.to_datetime(df['acq_date'])

df["month"] = df["datetime"].dt.month
df["week"] = df['datetime'].dt.isocalendar().week

# Group by month and count the number of fires
monthly_counts = df.groupby("month").size()

# Group by week and count the number of fires
weekly_counts = df.groupby("week").size()

In [ ]:
# Plotting
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Plotting monthly counts
monthly_counts.plot(kind='bar', ax=ax, color='skyblue')
ax.set_ylabel('Number of Fires', fontweight="bold")
ax.set_xlabel('Months', fontweight="bold")
ax.set_title('Monthly Fire Counts', fontweight="bold")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 8))

# Plotting weekly counts
weekly_counts.plot(kind='bar', ax=ax, color='skyblue')
ax.set_ylabel('Number of Fires', fontweight="bold")
ax.set_xlabel('Weeks', fontweight="bold")
ax.set_title('Weekly Fire Counts', fontweight="bold")
plt.show()

## Plotting Fire Radiative Power of these fires

In [ ]:
# Plotting
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Plotting monthly counts
df.groupby("week")["frp"].mean().plot(kind='line', ax=ax, color='skyblue')
ax.set_ylabel('Fire Radiative Power (MW)', fontweight="bold")
ax.set_xlabel('Weeks', fontweight="bold")
ax.set_title('Weekly Fire Counts', fontweight="bold")
plt.show()

## Adding vegetation types to fires

### How to look to landcover

Landcover data is generally available in the form of image where each pixel has a associated discreate number like 1 or 7 representing landcover class it belongs to like cropland or forest etc. Most landcover data covers major landcover class like Forest, Cropland, Shrubland, Grassland, Water, Snow/Ice, Wetland etc. In some case they also provide classes within a landcover like Forest 1, Forest 2 based on tree density, crown, height etc useful for different application. For identifying landcover for fires you can just group them into individual parent class.

Available Sources of Landcover
1. [ESA Landcover 300m from 1992 to 2019](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-land-cover?tab=form)
2. [ESA Landcover 100m from 2015 to 2019](https://land.copernicus.eu/global/products/lc), use the zenode link.
3. [ESA Landcover 10m from 2020 to 2021](https://esa-worldcover.org/en)



In [ ]:
# source https://zenodo.org/records/3939050
!wget https://zenodo.org/records/3939050/files/PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif -q

In [ ]:
landcover_mapping = {
  111:"Forest",
  112:"Forest",
  113:"Forest",
  114:"Forest",
  115:"Forest",
  116:"Forest",
  121:"Forest",
  122:"Forest",
  123:"Forest",
  124:"Forest",
  125:"Forest",
  126:"Forest",
  20:"Shrubland",
  30:"Grassland",
  40:"Cropland",
  90:"Peatland"
}

In [ ]:
def get_window(latitude, longitude, dataset):
    """
    Gets the closest index to a given lat, lon
    :param latitude: latitude
    :param longitude: longitude
    :param dataset: landcover dataset
    :return: list
    """
    xmin, xres, xskew, ymax, yskew, yres = dataset.GetGeoTransform()
    window = [
        math.floor((longitude - xmin) / xres),
        math.floor((latitude - ymax) / yres),
    ]
    return window


def get_raster_value(latitude, longitude, dataset, window_size=1):
    """
    Gets the raster value on a given lat, lon
    :param latitude: latitude
    :param longitude: longitude
    :param dataset: landcover dataset
    :return: indices
    """
    window = get_window(latitude, longitude, dataset)
    raster_array = dataset.ReadAsArray(window[0] - window_size//2, window[1] - window_size//2, window_size, window_size)
    mode_value = mode(raster_array, axis=None).mode
    if mode_value is None:
        return 0
    return mode_value

In [ ]:
# Get the landcover class for each coordinate
dataset = gdal.Open('/content/PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif')
df['landcover_class'] = [get_raster_value(lat, lon, dataset) for lat, lon in zip(df['latitude'], df['longitude'])]
dataset = None

In [ ]:
df["landcover_name"] = df["landcover_class"].apply(lambda x: landcover_mapping[x] if x in landcover_mapping else np.NaN)

## Estimates fire counts per vegetation types

In [ ]:
# Group by month and count the number of fires
fires_per_landcover = df.groupby("landcover_name").size()


# Plotting
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Plotting monthly counts
fires_per_landcover.plot(kind='bar', ax=ax, color='skyblue')
ax.set_ylabel('Number of Fires', fontweight="bold")
ax.set_xlabel('Landcover', fontweight="bold")
ax.set_title('Fire Counts based on vegetation type', fontweight="bold")
plt.show()

## Plotting Vegetation Fires over time

In [ ]:
forest_fires_per_week = df[df["landcover_name"]=="Forest"].groupby("week").size()

fig, ax = plt.subplots(1, 1, figsize=(15, 8))

# Plotting weekly counts
forest_fires_per_week.plot(kind='bar', ax=ax, color='skyblue')
ax.set_ylabel('Number of Forest Fires', fontweight="bold")
ax.set_xlabel('Weeks', fontweight="bold")
ax.set_title('Weekly Fire Counts', fontweight="bold")
plt.show()

In [ ]:
forest_fires_per_week = df[df["landcover_name"]=="Cropland"].groupby("week").size()

fig, ax = plt.subplots(1, 1, figsize=(15, 8))

# Plotting weekly counts
forest_fires_per_week.plot(kind='bar', ax=ax, color='skyblue')
ax.set_ylabel('Number of Crop Fires', fontweight="bold")
ax.set_xlabel('Weeks', fontweight="bold")
ax.set_title('Weekly Fire Counts', fontweight="bold")
plt.show()

## Plotting Fires on Map

In [ ]:
import geopandas as gpd

In [ ]:
#!wget https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_5m.zip

In [ ]:
!unzip /content/cb_2018_us_state_5m.zip -d ./

In [ ]:
gdf_usa = gpd.read_file("./cb_2018_us_state_5m.shp")
gdf_usa = gdf_usa.to_crs("EPSG:4326")
gdf_usa.head()

In [ ]:
gdf_california = gdf_usa[gdf_usa["NAME"]=="California"]
gdf_california

In [ ]:
geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

In [ ]:
gdf_california_fires = gpd.sjoin(gdf, gdf_california, how="inner", predicate="within")

In [ ]:
# Plot California shape
fig, ax = plt.subplots(figsize=(16, 10))
ax = gdf_california.plot(
    edgecolor='#64513B',
    facecolor='none',
    ax=ax)

gdf_california_fires.plot(
    markersize=gdf_california_fires['frp'],
    column='landcover_name',
    categorical=True,
    legend=True,
    ax=ax,
    alpha=0.5
)

ax.set_axis_off()
plt.show()